# 数据处理
## 1. 爬虫部分

### 1.1 计算爬取的总数据量（条）

In [1]:
import pandas as pd

filepath = "DataCrawler/MyData/Data_.csv"
tot_len = 0
for i in range(1, 29):
    pathl = filepath[0:-4]
    pathr = filepath[-4:]
    nowpath = pathl + str(i) + pathr
    # print(nowpath)
    data = pd.read_csv(nowpath)
    tot_len += len(data)
    # print(data)
print(tot_len)

120900


### 1.2 查看一个文件的数据

In [2]:
filepath = "DataCrawler/MyData/Data_1.csv"
data = pd.read_csv(filepath)
print(data)

       idiom1   idiom2                                           sentence
0        哀兵必胜     骄兵必败  有句话叫做骄兵必败，哀兵必胜。虽然我们队赢了这场比赛，但我们不能骄傲，否则下次输的一方一定是我们。
1        哀兵必胜  置之死地而后生                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
2        哀兵必胜     置之死地                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
3     置之死地而后生     置之死地                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
4        哀兵必胜     屡战屡败  因为牺牲的都是身边的亲友，感情很深，正和哀兵必胜的兵家奥义，可是碰上太平军就是屡战屡败，只能...
...       ...      ...                                                ...
7107     闭门思过     一败涂地            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7108     闭门思过     拱手让人            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7109     一败涂地     拱手让人            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7110     闭门思过     坐以待毙  在当今主流被各种门事件牵着走的时候，俺，也不坐以待毙，咱也来个门，啥门？闭门！——闭门思过、...
7111     闭门思过     得过且过  【光棍节的种过法】快乐法：一笑而过；认命法：得过且过；潇洒法：飘过；实诚法：路过；忏悔法：闭...

[7112 rows x 3 columns]


## 2. 打标签部分
### 2.1 将csv文件转成xlsx文件

In [5]:
import pandas as pd
def csv_to_xlsx_pd(fileFrom, fileTo):
    csv = pd.read_csv('DataCrawler/MyData/' + fileFrom, encoding='utf-8')
    csv.to_excel('DataCrawler/MyXlsx/' + fileTo, sheet_name='data', index = False)

for i in range(1, 29):
    fileFrom = "Data_" + str(i) + ".csv"
    fileTo = "Data_" + str(i) + ".xlsx"
    csv_to_xlsx_pd(fileFrom, fileTo)

——**漫长的打标签**——

## 3. 添加成语的举例`example`以及解释`explanation`数据字段
### 3.1 重构json文件的数据

In [1]:
import json

idiomDict = {}
# 读取idiom.json中的数据 重构
with open('idiom.json', 'r',encoding="utf-8") as f:
    lists = f.readlines()[0]
    preData = json.loads(lists)
    for data in preData:
        word = data["word"]
        idiomDict[word] = {}
        idiomDict[word]["explanation"] = data["explanation"]
        idiomDict[word]["example"] = data["example"]
# print(idiomDict) ############这里不要输出 否则JupyterNotebook会爆内存

### 3.2 添加字段并分割数据集

In [17]:
import pandas as pd
import numpy as np  
import re
import random
from sklearn.utils import shuffle

# 保存为csv文件
def saveCsvData(datalist, savepath):
    if len(datalist) == 0:
        return
    data = pd.DataFrame(datalist)
    data.to_csv(savepath, index=False,
                header=['idiom1', 'idiom2', 'explanation1', 'example1', 'explanation2', 'example2', 'label'])

# 获取两个成语的解释和举例
def getExplanationAndExample(idiom1, idiom2):
    dic1 = idiomDict.get(idiom1)
    dic2 = idiomDict.get(idiom2)
    if dic1 == None or dic2 == None:               # 在idiomDict中查找 若没有 则跳过
        return (None, None, None, None)
    example1 = dic1['example']
    example2 = dic2['example']
    example1 = re.sub('(～)+', idiom_i[0], example1) # 使用正则表达式 将成语加入到造句中
    example2 = re.sub('(～)+', idiom_i[1], example2)
    example1 = re.sub('(★.*)+', "", example1)       # 使用正则表达式 去除造句来源
    example2 = re.sub('(★.*)+', "", example2)
    example1 = re.sub('(（.*)+', "", example1)
    example2 = re.sub('(（.*)+', "", example2)
    explanation1 = dic1['explanation']
    explanation2 = dic2['explanation']
    return (explanation1, example1, explanation2, example2)
    
allData = []

# 遍历所有Xlsx文件 从idiomDict中添加字段
for page in range(1, 2):
    file_path = "Data_" + str(page) + ".xlsx"
    data_xlsx = pd.read_excel(file_path)
    # print(data_xlsx)
    data_list = np.array(data_xlsx)

    for data_i in data_list:
        idiom_i = [data_i[0], data_i[1]]
        explanation1, example1, explanation2, example2 = getExplanationAndExample(data_i[0], data_i[1])
        if explanation1 == None:
            continue
        idiom_i.append(explanation1)
        idiom_i.append(example1)
        idiom_i.append(explanation2)
        idiom_i.append(example2)
        idiom_i.append(data_i[3])
        
        allData.append(idiom_i)

        
# 添加若干不具有逻辑关系的成语 与打标签数据个数相同
tot = len(allData)
for i in range(tot):
    idiomList = list(idiomDict.keys())
    idiomNum = len(idiomList)
    idiom1 = idiomList[random.randint(0,idiomNum)]
    idiom2 = idiomList[random.randint(0,idiomNum)]
    explanation1, example1, explanation2, example2 = getExplanationAndExample(idiom1, idiom2)
    idiom = [idiom1, idiom2, explanation1, example1, explanation2, example2, 0]
    allData.append(idiom)

allData = shuffle(allData)
# 拿出10%的数据用来测试
test_proportion = 0.1
test_idx = int(len(allData) * test_proportion)
# 分割训练集和测试集
train_data = allData[test_idx:]
test_data = allData[:test_idx]

savepath = "IdiomData_train.csv"
saveCsvData(train_data, savepath)
print(len(train_data))
savepath = "IdiomData_test.csv"
saveCsvData(test_data, savepath)
print(len(test_data))

312
34


In [20]:
# 注意文件大的话 这里不要输出 否则JupyterNotebook会爆内存
filepath = "IdiomData_test.csv"
data = pd.read_csv(filepath)
print(data)

     idiom1  idiom2                                 explanation1  \
0      狼烟四起    生灵涂炭           狼烟古代边防报警时烧狼粪腾起的烟。四处都是报警的烟火，指边疆不平静。   
1      爱憎分明    拖泥带水                            憎恨。爱和恨的立场和态度十分鲜明。   
2      闲言长语     人急至                                    议人长短的唠叨话。   
3      安步当车    逍遥自在                   安安详，不慌忙；安步缓缓步行。以从容的步行代替乘车。   
4      唉声叹气    自言自语                            因伤感郁闷或悲痛而发出叹息的声音。   
5      白日见鬼    不寒而栗         大白天看见鬼。指工部四曹无事可做，非常清闲◇泛指事情离奇古怪或无中生有。   
6      刮骨抽筋    举一废百                                 比喻极力搜刮、榨取民财。   
7      马捉老鼠    鱼烂瓦解                                       比喻瞎忙乱。   
8      查无实据    添枝增叶                             查究起来，没有确实的根据或证据。   
9      无兄盗嫂    随声吠影                                    指无中生有的毁谤。   
10     此动彼应    无能为力                                   这里发动，那里响应。   
11     安然无恙    逍遥法外                    恙病。原指人平安没有疾病。现泛指事物平安未遭损害。   
12     碍手碍脚    一事无成                                  碍阻碍。妨碍别人做事。   
13     雕心雁爪    应答如响                             

**至此，数据集构建完成**